[Reference](https://levelup.gitconnected.com/monte-carlo-simulation-in-python-advanced-investment-risk-analysis-c28d4532b05b)

In [3]:
!python3 -m pip install eodhd -U

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from eodhd import APIClient

API_KEY = "<YOUR_KEY>"

api = APIClient(API_KEY)
df = api.get_historical_data("GSPC.INDX", "d", results=365)

print(df)

# Calculate daily returns
daily_returns = df["adjusted_close"].pct_change().dropna()
print(daily_returns)

In [5]:
# Simulation parameters
num_simulations = 1000
forecast_days = 365

# Initialise simulation array, all zeros
simulations = np.zeros((num_simulations, forecast_days))

# Simulate future paths
last_price = df["adjusted_close"].iloc[-1]
for i in range(num_simulations):
    cumulative_returns = np.random.choice(daily_returns, size=forecast_days, replace=True).cumsum()
    simulations[i, :] = last_price * (1 + cumulative_returns)

print(simulations)

In [6]:
# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(simulations.T, color="blue", alpha=0.025)
plt.title("Monte Carlo Simulation of Future Prices")
plt.xlabel("Day")
plt.ylabel("Price")
plt.show()

In [7]:
# Calculate daily returns
daily_returns = df["adjusted_close"].pct_change().dropna()

# Simulation parameters
initial_investment = 10000  # Initial investment amount
num_simulations = 1000  # Number of simulations
forecast_days = 365  # Investment horizon in days
desired_return = 0.10  # Desired return (10%)

# Calculate the average daily return
average_daily_return = daily_returns.mean()

# Calculate volatility as the standard deviation of daily returns
volatility = daily_returns.std()

print(f"Average Daily Return: {average_daily_return}")
print(f"Volatility: {volatility}")

In [8]:
daily_returns = np.log(df["adjusted_close"] / df["adjusted_close"].shift(1)).dropna()

In [9]:
# Simulating future returns
simulated_end_returns = np.zeros(num_simulations)
for i in range(num_simulations):
    random_returns = np.random.normal(average_daily_return, volatility, forecast_days)
    cumulative_return = np.prod(1 + random_returns)
    simulated_end_returns[i] = initial_investment * cumulative_return

# Calculate the final investment values
final_investment_values = simulated_end_returns

print(final_investment_values)

In [10]:
confidence_level = 0.95
sorted_returns = np.sort(final_investment_values)
index_at_var = int((1-confidence_level) * num_simulations)
var = initial_investment - sorted_returns[index_at_var]
conditional_var = initial_investment - sorted_returns[:index_at_var].mean()

print(f"Value at Risk (95% confidence): £{var:,.2f}")
print(f"Expected Tail Loss (Conditional VaR): £{conditional_var:,.2f}")

In [11]:
num_success = np.sum(final_investment_values >= initial_investment * (1 + desired_return))
probability_of_success = num_success / num_simulations

print(f"Probability of achieving at least a {desired_return*100}% return: {probability_of_success*100:.2f}%")

In [12]:
plt.figure(figsize=(10, 6))
plt.hist(final_investment_values, bins=50, alpha=0.75)
plt.axvline(
      initial_investment * (1 + desired_return),
      color="r",
      linestyle="dashed",
      linewidth=2,
      )
plt.axvline(initial_investment - var, color="g", linestyle="dashed", linewidth=2)
plt.title("Distribution of Final Investment Values")
plt.xlabel("Final Investment Value")
plt.ylabel("Frequency")
plt.show()

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from eodhd import APIClient
import config as cfg

api = APIClient(cfg.API_KEY)


def get_ohlc_data():
    df = api.get_historical_data("GSPC.INDX", "d", results=365)
    return df


if __name__ == "__main__":
    df = get_ohlc_data()

    # Calculate daily returns
    daily_returns = df["adjusted_close"].pct_change().dropna()

    # Simulation parameters
    initial_investment = 10000  # Initial investment amount
    num_simulations = 1000  # Number of simulations
    forecast_days = 365  # Investment horizon in days
    desired_return = 0.10  # Desired return (10%)

    # Calculate the average daily return
    average_daily_return = daily_returns.mean()

    # Calculate volatility as the standard deviation of daily returns
    volatility = daily_returns.std()

    print(f"Average Daily Return: {average_daily_return}")
    print(f"Volatility: {volatility}")

    # Simulating future returns
    simulated_end_returns = np.zeros(num_simulations)
    for i in range(num_simulations):
        random_returns = np.random.normal(
            average_daily_return, volatility, forecast_days
        )
        cumulative_return = np.prod(1 + random_returns)
        simulated_end_returns[i] = initial_investment * cumulative_return

    # Calculate the final investment values
    final_investment_values = simulated_end_returns

    # Calculate Value at Risk (VaR) and Expected Tail Loss (Conditional VaR)
    confidence_level = 0.95
    sorted_returns = np.sort(final_investment_values)
    index_at_var = int((1 - confidence_level) * num_simulations)
    var = initial_investment - sorted_returns[index_at_var]
    conditional_var = initial_investment - sorted_returns[:index_at_var].mean()

    print(f"Value at Risk (95% confidence): £{var:,.2f}")
    print(f"Expected Tail Loss (Conditional VaR): £{conditional_var:,.2f}")

    num_success = np.sum(
        final_investment_values >= initial_investment * (1 + desired_return)
    )
    probability_of_success = num_success / num_simulations

    print(
        f"Probability of achieving at least a {desired_return*100}% return: {probability_of_success*100:.2f}%"
    )

    plt.figure(figsize=(10, 6))
    plt.hist(final_investment_values, bins=50, alpha=0.75)
    plt.axvline(
        initial_investment * (1 + desired_return),
        color="r",
        linestyle="dashed",
        linewidth=2,
    )
    plt.axvline(initial_investment - var, color="g", linestyle="dashed", linewidth=2)
    plt.title("Distribution of Final Investment Values")
    plt.xlabel("Final Investment Value")
    plt.ylabel("Frequency")
    plt.show()

    """
    # Plotting the results
    plt.figure(figsize=(10, 6))
    plt.plot(simulations.T, color="blue", alpha=0.025)
    plt.title("Monte Carlo Simulation of Future Prices")
    plt.xlabel("Day")
    plt.ylabel("Price")
    plt.show()
    """